In [1]:
#default_exp rest

In [2]:
#export

import requests
import json
from nifi_api.environment import NifiEndpoint, Credentials
from nifi_api.tools import custom_response, get_valid_names_in_connector

In [3]:
#hide
%load_ext autoreload
%autoreload 2

# REST
> Handles the states of the Nifi API 

## Processors
> Handles the route **Processors** 

In [4]:
#export


class Processor:

    nifi_api = NifiEndpoint.processors

    def __init__(self, processor_id: str) -> None:
        self.processor_id = processor_id

    def get(self) -> tuple:
        url = self.nifi_api + self.processor_id
        res = requests.get(url, auth=Credentials.credentials)
        return custom_response(res)

    def get_state(self) -> None:
        self.state, _ = self.get()
        self.version = self.state['revision']['version']

    def update_run_status(self, state) -> tuple:
        """
Change the status of the processor.

 Attributes
----------
state : str
     The possible values are: "RUNNING", "STOPPED", "DISABLED".

 Returns
----------
 custom_response : tuple
   The response as defined in tools module
"""

        self.get_state()

        data = {
            "revision": {
                #   'clientId': self.client_id,
                'version': self.version
            },
            "state": state,
            "disconnectedNodeAcknowledged": True
        }

        url = self.nifi_api + f"{self.processor_id}/run-status"

        res = requests.put(url,
                           data=json.dumps(data),
                           auth=Credentials.credentials,
                           headers={'content-type': 'application/json'})
        return custom_response(res)

In [6]:
#test
# Uses the group processor "Test API"
# in a Cloudera session

p = Processor('cc555e79-0177-1000-ffff-ffffac3be751')

In [11]:
_, r = p.update_run_status(state="STOPPED")

In [15]:
assert  r.status_code == 200

## FlowFile Queues
> Handles the route **Flowfiles**

In [16]:
#export


class Flowfiles:

    def __init__(self, connection_id: str) -> None:
        self.connection_id = connection_id

    def post_listing_request(self) -> None:
        url = NifiEndpoint.flowfile_queues + f"{self.connection_id}/listing-requests"
        res = requests.post(url, auth=Credentials.credentials)
        self.listing_request = custom_response(res)[0]
        self.listing_request_id = self.listing_request["listingRequest"]['id']

    def get_list_queue(self) -> None:
        self.post_listing_request()
        endpoint = f"{self.connection_id}/listing-requests/{self.listing_request_id}"
        url = NifiEndpoint.flowfile_queues + endpoint
        res = requests.get(url, auth=Credentials.credentials)
        self.list_queue, _ = custom_response(res)

    def get_ids(self) -> None:
        self.get_list_queue()
        self.ids = get_valid_names_in_connector(self.list_queue)

    def equals(self, flowfile) -> bool:
        return set(self.ids) == set(flowfile.ids)

In [19]:
#test

# This test is using the Test API processor in the Cloudera session  Steps:

# 1. Turn all the processors Off. Generate a flowfile by turning on
# processor "One". Read it as follows:
test_flowfiles1 = Flowfiles("cc549c6e-0177-1000-ffff-ffffb5d2aba2")
test_flowfiles1.get_ids()

In [20]:
#test


# 3. Turn ON processor "two". Read the corresponding flowfile 
#  in the success relationship

test_flowfiles2 = Flowfiles("cc557339-0177-1000-0000-000001afcc7b")
test_flowfiles2.get_ids()

In [21]:
# 4. Compare the two ids. They must coincide

assert test_flowfiles2.equals(test_flowfiles1)

In [22]:
# 4. Turn ON processor "three". Read the corresponding flowfile
#    in the success relationship

test_flowfiles3 = Flowfiles("51ab3b24-084f-1309-0000-00001946f2c7")
test_flowfiles3.get_ids()

In [23]:
assert test_flowfiles3.equals(test_flowfiles1)
assert test_flowfiles2.equals(test_flowfiles3)

In [24]:
get_valid_names_in_connector(test_flowfiles2.list_queue)

[]

In [26]:
from nbdev.export import notebook2script
notebook2script()

Converted 01_environment.ipynb.
Converted 02_rest.ipynb.
Converted 03_dataflow.ipynb.
Converted 04_source_to_refined.ipynb.
Converted 09_tools.ipynb.
Converted index.ipynb.
